In [6]:
import platform
print(f"platform.python_version(): {platform.python_version()}")

import os
import scipy
import numpy as np
from glob import glob
from tqdm import tqdm
import json
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation

# 確認GPU是否可用
gpus = tf.config.experimental.list_physical_devices("GPU")
print(f"tf.config.list_physical_devices('GPU'): {tf.config.list_physical_devices('GPU')}")

platform.python_version(): 3.9.13
tf.config.list_physical_devices('GPU'): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
npy_file_path_list: list = glob(f"..\\data\\mfccEduVer\\*.npy")
print(f"type(npy_file_path_list): {type(npy_file_path_list)}")
print(f"len(npy_file_path_list): {len(npy_file_path_list)}")
# npy_file_path_list

type(npy_file_path_list): <class 'list'>
len(npy_file_path_list): 45167


In [10]:
mfcc_matrix_list = list()

for npy_file_path in tqdm(npy_file_path_list):
    mfcc_matrix = np.load(file=npy_file_path)
    mfcc_matrix_list.append(mfcc_matrix)

mfcc_matrix_list = np.array(mfcc_matrix_list)
print(f"type(mfcc_matrix_list): {type(mfcc_matrix_list)}")
print(f"mfcc_matrix_list.shape: {mfcc_matrix_list.shape}")  
# mfcc_matrix_list

100%|██████████| 45167/45167 [00:07<00:00, 6157.03it/s]


type(mfcc_matrix_list): <class 'numpy.ndarray'>
mfcc_matrix_list.shape: (45167, 13, 44)
